# ETL

Este notebook realiza la:
1. **Extracción:** de los datos de la API de Banxico e INEGI.
2. **Transformación:** utilizando DatFrames de Pandas.
3. **Carga:** a un bucket de AWS S3.

## Librerías

In [4]:
import yaml
import requests
import json
import pandas as pd
import boto3
from botocore.exceptions import ClientError
from io import StringIO

## Importa parámetros de configuración

Tokens generados en:
- Banxico: https://www.banxico.org.mx/SieAPIRest/service/v1/token
- INEGI: https://www.inegi.org.mx/app/api/denue/v1/tokenVerify.aspx

In [5]:
with open('config.yaml') as f:
    config = yaml.safe_load(f)
    
BANXICO_API_TOKEN = config["banxico"]["token"]
INEGI_API_TOKEN = config["inegi"]["token"]
BUCKET_NAME = config["aws"]["bucket_name"]
DATABASE_NAME = config["aws"]["database_name"]

## Extrae datos Banxico

Definimos las series de tiempo que queremos extraer de la API de Banxico.

In [6]:
serie_tipo_cambio = "SF43718"
serie_tasa_interes = "SF61745"

Extraemos la metadata de las series de tiempo.

In [7]:
url_banxico_metadata = f"https://www.banxico.org.mx/SieAPIRest/service/v1/series/{serie_tipo_cambio},{serie_tasa_interes}?token={BANXICO_API_TOKEN}"
response_banxico_metadata = requests.get(url_banxico_metadata)

if response_banxico_metadata.status_code == 200:
    banxico_metadata = response_banxico_metadata.json()
    print(json.dumps(banxico_metadata, indent=2))
else:
    print(f"Error: {response_banxico_metadata.status_code} - {response_banxico_metadata.text}")

{
  "bmx": {
    "series": [
      {
        "idSerie": "SF43718",
        "titulo": "Tipo de cambio                                          Pesos por d\u00f3lar E.U.A. Tipo de cambio para solventar obligaciones denominadas en moneda extranjera Fecha de determinaci\u00f3n (FIX)",
        "fechaInicio": "12/11/1991",
        "fechaFin": "19/03/2025",
        "periodicidad": "Diaria",
        "cifra": "Tipo de Cambio",
        "unidad": "Pesos por D\u00f3lar",
        "versionada": false
      },
      {
        "idSerie": "SF61745",
        "titulo": "Tasa objetivo",
        "fechaInicio": "21/01/2008",
        "fechaFin": "20/03/2025",
        "periodicidad": "Diaria",
        "cifra": "Porcentajes",
        "unidad": "Sin Unidad",
        "versionada": false
      }
    ]
  }
}


Extraemos los datos de la serie de tiempo

In [8]:
url_banxico_data = f"https://www.banxico.org.mx/SieAPIRest/service/v1/series/{serie_tipo_cambio},{serie_tasa_interes}/datos?token={BANXICO_API_TOKEN}"
response_banxico_data = requests.get(url_banxico_data)

if response_banxico_data.status_code == 200:
    banxico_data = response_banxico_data.json()
    print("Extracción de datos exitosa")
else:
    print(f"Error: {response_banxico_data.status_code} - {response_banxico_data.text}")

Extracción de datos exitosa


### Transformación de datos Banxico

In [9]:
tipo_de_cambio = pd.DataFrame(next(item for item in banxico_data["bmx"]["series"] if item["idSerie"] == serie_tipo_cambio)["datos"])
tipo_de_cambio["fecha"] = pd.to_datetime(tipo_de_cambio["fecha"], dayfirst=True).dt.strftime("%Y-%m-%d")
tipo_de_cambio["dato"] = tipo_de_cambio["dato"].astype(float)
tipo_de_cambio = tipo_de_cambio.rename(columns={"fecha": "timestamp", "dato": "tipo_de_cambio"})

tasa_de_interes = pd.DataFrame(next(item for item in banxico_data["bmx"]["series"] if item["idSerie"] == serie_tasa_interes)["datos"])
tasa_de_interes["fecha"] = pd.to_datetime(tasa_de_interes["fecha"], dayfirst=True).dt.strftime("%Y-%m-%d")
tasa_de_interes["dato"] = tasa_de_interes["dato"].astype(float)
tasa_de_interes = tasa_de_interes.rename(columns={"fecha": "timestamp", "dato": "tasa_de_interes"})

In [10]:
print(tipo_de_cambio.head())
print(tasa_de_interes.head())

    timestamp  tipo_de_cambio
0  1991-11-12          3.0735
1  1991-11-13          3.0712
2  1991-11-14          3.0718
3  1991-11-15          3.0684
4  1991-11-18          3.0673
    timestamp  tasa_de_interes
0  2008-01-21              7.5
1  2008-01-22              7.5
2  2008-01-23              7.5
3  2008-01-24              7.5
4  2008-01-25              7.5


## Extrae datos INEGI

Definimos las series de tiempo que queremos extraer de la API de INEGI.

In [11]:
serie_inflacion = "910399"

Extraemos la metadata de las series de tiempo.

In [12]:
url_inegi_metadata = f"https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/CL_INDICATOR/{serie_inflacion}/es/BIE/2.0/{INEGI_API_TOKEN}?type=json"

response_inegi_metadata = requests.get(url_inegi_metadata)

if response_inegi_metadata.status_code == 200:
    inegi_metadata = response_inegi_metadata.json()
    print(json.dumps(inegi_metadata, indent=2))
else:
    print(f"Error: {response_inegi_metadata.status_code} - {response_inegi_metadata.text}")

{
  "id": "CL_INDICATOR",
  "agencyID": "INEGI",
  "version": "1.0",
  "lang": "es",
  "CODE": [
    {
      "value": "910399",
      "Description": "\u00cdndice general"
    }
  ]
}


Extraemos los datos de la serie de tiempo

In [13]:
url_inegi_data = f"https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/{serie_inflacion}/es/0700/false/BIE/2.0/{INEGI_API_TOKEN}?type=json"

response_inegi_data = requests.get(url_inegi_data)

if response_inegi_data.status_code == 200:
    inegi_data = response_inegi_data.json()
    print("Extracción de datos exitosa")
else:
    print(f"Error: {response_inegi_data.status_code} - {response_inegi_data.text}")

Extracción de datos exitosa


### Transformación de datos INEGI

In [14]:
inflacion = pd.DataFrame(inegi_data["Series"][0]["OBSERVATIONS"])

inflacion["TIME_PERIOD"] =  pd.to_datetime(inflacion["TIME_PERIOD"], format="%Y/%m").dt.strftime("%Y-%m-%d")
inflacion["OBS_VALUE"] = pd.to_numeric(inflacion["OBS_VALUE"], errors="coerce")

inflacion = inflacion[["TIME_PERIOD", "OBS_VALUE"]].dropna()

inflacion = inflacion.rename(columns={"TIME_PERIOD": "timestamp", "OBS_VALUE": "inflacion"})

In [15]:
print(inflacion.head())

    timestamp  inflacion
0  2025-02-01       0.28
1  2025-01-01       0.29
2  2024-12-01       0.38
3  2024-11-01       0.44
4  2024-10-01       0.55


## Carga de datos en AWS S3

Abrimos un cliente de S3 y creamos el bucket donde guardaremos los datos.

In [16]:
session = boto3.Session(profile_name='arquitectura')
s3 = session.client('s3')

try:
    # Verifica si el bucket ya existe
    s3.head_bucket(Bucket=BUCKET_NAME)
    print(f"El bucket '{BUCKET_NAME}' ya existe.")
except ClientError as e:
    # Si el bucket no existe, lo crea
    if e.response['Error']['Code'] == '404':
        s3.create_bucket(Bucket=BUCKET_NAME)
        print(f"El bucket '{BUCKET_NAME}' ha sido creado.")
    else:
        print(f"Error al verificar o crear el bucket: {e}")

El bucket 'itam-analytics-ferlango' ya existe.


Cargamos los dataframes en memoria como archivos CSV para subirlos a S3.

In [21]:
def upload_df_to_s3(df, bucket, folder, file_name, extension):
    """
    Sube un DataFrame a S3 en la ruta: {folder}/raw/{file_name}/{file_name}{extension}.
    
    Parámetros:
      - df: DataFrame a subir.
      - bucket: Nombre del bucket de S3.
      - folder: Carpeta raíz (por ejemplo, el nombre de la base de datos, "econ").
      - file_name: Nombre base del archivo (sin extensión).
      - extension: Extensión del archivo, por ejemplo ".csv".
    """
    csv_buffer = StringIO()
    df.to_csv(csv_buffer, index=False)
    
    key = f"{folder}/raw/{file_name}/{file_name}{extension}"
    s3.put_object(Bucket=bucket, Key=key, Body=csv_buffer.getvalue().encode("utf-8"))
    print(f"Archivo '{file_name}{extension}' subido a s3://{bucket}/{key}")

In [22]:
upload_df_to_s3(tipo_de_cambio, BUCKET_NAME, DATABASE_NAME, "tipo_de_cambio", ".csv")
upload_df_to_s3(tasa_de_interes, BUCKET_NAME, DATABASE_NAME, "tasa_de_interes", ".csv")
upload_df_to_s3(inflacion, BUCKET_NAME, DATABASE_NAME, "inflacion", ".csv")

Archivo 'tipo_de_cambio.csv' subido a s3://itam-analytics-ferlango/econ/raw/tipo_de_cambio/tipo_de_cambio.csv
Archivo 'tasa_de_interes.csv' subido a s3://itam-analytics-ferlango/econ/raw/tasa_de_interes/tasa_de_interes.csv
Archivo 'inflacion.csv' subido a s3://itam-analytics-ferlango/econ/raw/inflacion/inflacion.csv
